In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix, hamming_loss

In [2]:
df = pd.read_csv("../Datasets/flavornet_dataset.csv")
df.dropna(inplace=True) # Drop N/A values

In [3]:
# Clean up the odor names by removing whitespace and commas

odors_series = df['Odor']
odors_series = odors_series.str.split(",")

for index in odors_series.index:
    # Clean up the odor names for each molecule
    odors_series[index] = [odor.strip().strip(',').lower() for odor in odors_series[index]]

In [4]:
all_odors = [] # List of all the odors present in the dataset
def get_labels_counts(array):
    for label in array:
        if label not in all_odors:
            all_odors.append(label)

odors_series.apply(get_labels_counts)

mlb = MultiLabelBinarizer() # Encoding the labels as boolean arrays
encoding = mlb.fit_transform(odors_series.tolist())
classes = mlb.classes_

In [5]:
# Need to add it to the data frame because we shuffle before splitting
encoding_df = pd.DataFrame(encoding, columns=classes)
df.reset_index(drop=True, inplace=True)
encoding_df.reset_index(drop=True, inplace=True)
final_df = pd.concat([df, encoding_df], axis=1) # Adding the matrix as columns to the data frame

# final_df is now the data frame to apply algorithms on

In [6]:
# Create training and test sets
train, test = train_test_split(final_df, test_size=0.25, shuffle=True)

labels_to_drop = all_odors.copy()
labels_to_drop += ['Odorant', 'Odor', 'SMILES representation']

x_train = train.drop(labels=labels_to_drop, axis=1)
y_train = train.loc[:, all_odors]

x_test = test.drop(labels=labels_to_drop, axis=1)
y_test = test.loc[:, all_odors]

In [7]:
# Fitting a random forest to the data #

model = RandomForestClassifier(class_weight='balanced')
model.fit(x_train, y_train)
y_predict = model.predict(x_test)

print("Hamming loss = ", hamming_loss(y_test, y_predict))

Hamming loss =  0.01051545705633756
